In [ ]:
import sys
sys.path.insert(0, "/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/saga")

In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H
from astropy.table import join
from SAGA.utils.distance import z2v

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
sats = FitsTable("/home/yymao/Dropbox/Academia/Collaborations/SAGA/Temp/paper2-preliminary-data-products/saga_sats_latest.fits").read()
sats = Query(saga.host_catalog.construct_host_query("paper2_complete")).filter(sats)

In [ ]:
sats.sort(["HOST_RA", "r_mag"])

In [ ]:
from SAGA.objects.cuts import _known_telnames as pre_exist_tel

In [ ]:
pre_exist_tel

In [ ]:
def format_tel(s):
    s = s.split("+")
    if len(s) == 1:
        return s[0]
    exist = sorted(pre_exist_tel.intersection(s))
    if not exist:
        return sorted(s)[0] + "+"
    
    while exist:
        if len(exist) == 1:
            return exist[0]
        if 'NSA' in exist:
            exist.remove("NSA")
            continue
        if 'GAMA' in exist:
            exist.remove("GAMA")
            continue
        if "SDSS" in exist:
            return "SDSS+"
        return sorted(exist)[0] + "+"

In [ ]:
def char_add(*args):
    args_iter = iter(args)
    a = next(args_iter)
    for b in args_iter:
        a = np.char.add(a, b)
    return a

In [ ]:
np.char.add(np.char.upper(sats["survey"]), np.char.add("-", sats["OBJID"].astype(str)))

In [ ]:
t = sats['HOST_COMMON_NAME', "OBJID", "survey", 'RA', "DEC", 'RHOST_KPC', 'SPEC_Z', "HOST_VHOST", 'r_mag', 'Mr', 'gr', 'sb_r', 'log_sm', "SPEC_REPEAT"]


t["OBJID"] = np.where(
    t["survey"] == "sdss", 
    t["OBJID"], 
    np.where(
        t["survey"] == "decals", 
        char_add("LS-", (sats["OBJID"] // int(1e13)).astype(str), "-", (sats["OBJID"] % int(1e13)).astype(str)),
        char_add(np.char.upper(sats["survey"]), "-", sats["OBJID"].astype(str)),
    ),
)
del t["survey"]
t["SPEC_REPEAT"] = list(map(format_tel, t["SPEC_REPEAT"]))
t['RA'].format = '%.4f'
t['DEC'].format = '%.4f'
t['RHOST_KPC'].format = '%d'
t["SPEC_Z"] = z2v(t["SPEC_Z"]) - t["HOST_VHOST"]
t["SPEC_Z"].format = '%.d'
del t["HOST_VHOST"]
t['r_mag'].format = '%.2f'
t['Mr'].format = '%.1f'
t['gr'].format = '%.2f'
t['sb_r'].format = '%.2f'
t['log_sm'].format = '%.1f'

t.rename_column('HOST_COMMON_NAME', 'Host Name')
t.rename_column('RA', 'R.A.')
t.rename_column('DEC', 'Dec.')
t.rename_column('SPEC_Z', '%\Delta v$')
t.rename_column('RHOST_KPC', '$d$')
t.rename_column('r_mag', 'r_o')
t.rename_column('Mr', r'M_{r,o}')
t.rename_column('gr', r'(g-r)_o}$')
t.rename_column('sb_r', r'\mu_{\mathrm{eff},r_o}$')
t.rename_column('log_sm', r'M_*')
t.rename_column('SPEC_REPEAT', 'Tel.')

In [ ]:
print(len(t))

In [ ]:
from io import StringIO
o = StringIO()
t.write(o, format="ascii.aastex")
print(o.getvalue())